In [1]:
from model_wrapper import CombinedLayoutNetPersp
import torch
import pytorch_lightning as pl
from torch import nn
import cv2

In [2]:
combined_model = CombinedLayoutNetPersp()
combined_model.load_component_weights(
    encoder_checkpoint_path='./ckpt/pre_encoder.pth',
    edge_decoder_checkpoint_path='./ckpt/pre_edg_decoder.pth',
    corner_decoder_checkpoint_path='./ckpt/pre_cor_decoder.pth',
    type_decoder_checkpoint_path='./ckpt/pre_type_decoder.pth',
)
combined_model = combined_model.eval()
combined_model

Loading encoder weights
Loading edge decoder weights
Loading corner decoder weights
Loading type decoder weights


CombinedLayoutNetPersp(
  (encoder): Encoder(
    (convs): ModuleList(
      (0): Sequential(
        (0): Sequential(
          (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Sequential(
        (0): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (3): Sequential(
        (0): Sequential(
          (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding

In [3]:
def save_model_checkpoint(model, ckpt_path):
    trainer = pl.Trainer(accelerator='cpu')
    trainer.strategy.connect(model)
    trainer.save_checkpoint(ckpt_path)

In [4]:
save_model_checkpoint(combined_model, './ckpt/combined_model.ckpt')

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/richard/miniconda3/envs/layoutnet-perspective/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


In [5]:
combined_model = CombinedLayoutNetPersp.load_from_checkpoint('./ckpt/combined_model.ckpt')
combined_model = combined_model.eval()

In [6]:
combined_model.to_onnx(
    './ckpt/combined_model.onnx',
    verbose=True, 
    export_params=True,
    input_names=['input'],
    output_names=['edge', 'corner', 'type'],
    dynamic_axes={
        'input' : {0 : 'batch_size'},
        'edge' : {0 : 'batch_size'},
        'corner' : {0 : 'batch_size'},
        'type' : {0 : 'batch_size'},
    }
)

Exported graph: graph(%input : Float(*, 3, 512, 512, strides=[786432, 262144, 512, 1], requires_grad=0, device=cpu),
      %encoder.convs.0.0.0.weight : Float(32, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cpu),
      %encoder.convs.0.0.0.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %encoder.convs.1.0.0.weight : Float(64, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=1, device=cpu),
      %encoder.convs.1.0.0.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %encoder.convs.2.0.0.weight : Float(128, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %encoder.convs.2.0.0.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %encoder.convs.3.0.0.weight : Float(256, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=1, device=cpu),
      %encoder.convs.3.0.0.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %encoder.convs.4.0.0.weight : Float(512, 256, 3, 3, strides=[2304, 9, 3, 1], requires

In [ ]:
# TODO: now create wrapper model around combined model that will perform basic result preprocessing and postprocessing (flip, etc)
#   (and preprocessing, if needed)

### CoreML Export

In [ ]:
# import os
# os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

# import coremltools as ct

In [7]:
# example = torch.rand_like(combined_model.example_input_array)

combined_model = combined_model.eval()

traced_model = combined_model.to_torchscript(method='trace')

In [ ]:
# model = ct.convert(
#     traced_model,
#     convert_to="mlprogram",
#     inputs=[ct.TensorType(shape=example.shape)]
#  )

In [ ]:
# model.save('./ckpt/combined_model.mlpackage')

In [10]:
with torch.no_grad():
    edge_tensor, corner_tensor, type_tensor = traced_model(combined_model.example_input_array)

edge_tensor.shape, corner_tensor.shape, type_tensor.shape

(torch.Size([1, 3, 512, 512]),
 torch.Size([1, 8, 512, 512]),
 torch.Size([1, 11]))

## Sample Inference

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch

In [ ]:
# image = Image.open('./images/room4.jpg')
# image = Image.open('./images/room0.jpg')
# image = Image.open('./images/room9.jpg')
# image = Image.open('./images/room5.jpg')
# image = Image.open('./images/room6.jpg')
# image = Image.open('./images/room_0_2.jpeg')
# image = Image.open('./images/room_0_3.jpeg')
image = Image.open('./images/room5_2.jpeg')

input_image = np.array(image.resize((512, 512)), np.float32) / 255

plt.imshow(input_image)
plt.axis('off');

In [ ]:
x_img = input_image.transpose([2, 0, 1]) # HWC -> CHW

In [ ]:
x = torch.FloatTensor(x_img)
# Add batch dimension to make it NCHW
x = x.unsqueeze(0)
x.shape

In [ ]:
with torch.no_grad():
    [edg_de_list, cor_de_list, type_tensor] = combined_model(x)

In [ ]:
edg_de_list[-1].shape

In [ ]:
edg_tensor = torch.sigmoid(edg_de_list[-1])
cor_tensor = torch.sigmoid(cor_de_list[-1])

print(f'edg_tensor shape: {edg_tensor.shape}\ncor_tensor shape: {cor_tensor.shape}')

# # Recover the effect from augmentation
# edg_img = augment_undo(edg_tensor.cpu().numpy(), aug_type)
# cor_img = augment_undo(cor_tensor.cpu().numpy(), aug_type)

In [ ]:
type_tensor = type_tensor.softmax(1)
print(f'Type tensor: {type_tensor}\nRoom type: {type_tensor.argmax()}')

In [ ]:
edg_tensor_image = edg_tensor.squeeze().permute((1, 2, 0)) # NCHW -> HWC
print(edg_tensor_image.shape)

plt.imshow(edg_tensor_image)
plt.axis('off')
plt.title('Edge map');

In [ ]:
corner_tensor = cor_tensor.squeeze().permute((1, 2, 0))
for channel_idx in range(0, corner_tensor.shape[-1]):
    plt.figure()
    plt.imshow(corner_tensor[:, :, channel_idx])
    plt.axis('off')
    plt.title(f'Channel {channel_idx}')

In [ ]:
# TODO: blend edges, corners on image
image_overlay = np.array(image.copy().resize((512, 512)))

edge_mask = (edg_tensor_image.numpy() * 255).astype(np.uint8)
image_overlay = cv2.addWeighted(
    image_overlay, 
    1.0,
    edge_mask, 
    1.0,
    0
)

for channel_idx in range(0, corner_tensor.shape[-1]):
    corner_map = (corner_tensor[:, :, channel_idx].numpy() * 255).astype(np.uint8)
    corner_image = np.zeros_like(image_overlay)
    corner_image[:, :, 0] = corner_map
    corner_image[:, :, 0] = corner_map
    corner_image[:, :, 2] = corner_map
    image_overlay = cv2.addWeighted(
        image_overlay, 
        1.0,
        corner_image, 
        1.0,
        0
    )


plt.imshow(image_overlay)
plt.axis('off');

In [ ]:
corner_map[..., None].shape

In [ ]:
image_overlay.dtype

In [ ]:
edge_mask[..., None].shape

In [ ]:
corner_tensor.mean(0)[..., 1].shape